In [1]:
import numpy as np
import pandas as pd
import bcolz
import time
import logging
import datetime

import sys
sys.path.append('..')

from bcolzutils import *
from util import *

import keras.backend as K
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import regularizers
from keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger, LearningRateScheduler
from keras import optimizers
from keras import regularizers

from keras.applications.vgg19 import VGG19
from keras.applications.vgg19 import preprocess_input as vgg19_preprocess_input

from keras.applications.mobilenet import MobileNet
from keras.applications.mobilenet import preprocess_input as mobile_preprocess_input

import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config = config)

Using TensorFlow backend.
/home/tutysara/anaconda2/envs/dog-project/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
arch = "caffe_vgg19"
basedir="/media/hdd/datastore/t4sa"
#basedir="/home/tutysara/src/myprojects/dog-project/dogImages"

percent = 0.005
#percent = 1
epochs=5
#epochs=15
#num_classes = 133
num_classes = 3 
#batch_size = 48
batch_size = 64
lr=1e-3
momentum=0.9
l2_weight_decay = 1e-5
test_prefix=""

def lr_schedule(epoch):
    """ divides the lr by 10 every 5 epochs"""
    n = (epoch + 1) // 5
    return lr / (10 ** n)

if percent < 1:
    test_prefix = "test_"

d = datetime.datetime.today()

model_path = f'saved_models/{test_prefix}all_layers_{arch}_weights.hdf5'
loss_history_csv_name = f'{test_prefix}all_layers.{arch}_loss_history.csv'
test_result = f'saved_models/{test_prefix}all_layers_{arch}_result.npz'
log_filename=f"all_layer_train_{arch}_{d.year}-{d.month}-{d.day}-{d.hour}.{d.minute}.{d.second}_{test_prefix}.log"


logging.basicConfig(level='DEBUG',
                    handlers=[logging.FileHandler(log_filename),
                              logging.StreamHandler()])
log = logging.getLogger(__name__)


log.debug("fine tune all layers")
log.debug("using all_model_weight_path :" + model_path)
log.debug("using test_result :" + test_result)
log.debug("using loss_history_csv_name :" + loss_history_csv_name)

train_name = basedir + '/pp_train_data'
valid_name = basedir + '/pp_valid_data'
test_name = basedir + '/pp_test_data'

temp_dir = "/tmp/" 

DEBUG:__main__:fine tune all layers
DEBUG:__main__:using all_model_weight_path :saved_models/test_all_layers_caffe_vgg19_weights.hdf5
DEBUG:__main__:using test_result :saved_models/test_all_layers_caffe_vgg19_result.npz
DEBUG:__main__:using loss_history_csv_name :test_all_layers.caffe_vgg19_loss_history.csv


In [3]:
%ls -l {basedir}

total 66038528
drwxrwxrwx 1 root root           0 Feb 12 21:36 bottleneck_features_mobilenet_test_data.bclz/
drwxrwxrwx 1 root root           0 Feb 12 21:36 bottleneck_features_mobilenet_test_labels.bclz/
drwxrwxrwx 1 root root           0 Feb 21 20:50 bottleneck_features_mobilenet_test_y_pred.bclz/
drwxrwxrwx 1 root root           0 Feb 21 20:50 bottleneck_features_mobilenet_test_y_true.bclz/
drwxrwxrwx 1 root root           0 Feb 12 21:49 bottleneck_features_mobilenet_train_data.bclz/
drwxrwxrwx 1 root root           0 Feb 12 21:49 bottleneck_features_mobilenet_train_labels.bclz/
drwxrwxrwx 1 root root           0 Feb 12 21:23 bottleneck_features_mobilenet_valid_data.bclz/
drwxrwxrwx 1 root root           0 Feb 12 21:23 bottleneck_features_mobilenet_valid_labels.bclz/
drwxrwxrwx 1 root root           0 Feb 23 23:50 bottleneck_features_vgg19_test_data.bclz/
drwxrwxrwx 1 root root           0 Feb 23 23:50 bottleneck_features_vgg19_test_labels.bclz/
drwxrwxrwx 1 root root           0 Ma

In [4]:
# read bcolz data
bclz_valid_data = bcolz.carray(rootdir= valid_name+'_data.bclz', mode='r')
bclz_test_data = bcolz.carray(rootdir= test_name + '_data.bclz', mode='r')
bclz_train_data = bcolz.carray(rootdir= train_name+ '_data.bclz', mode='r')


bclz_valid_labels = bcolz.carray(rootdir= valid_name+'_labels.bclz', mode='r')
bclz_test_labels = bcolz.carray(rootdir= test_name + '_labels.bclz', mode='r')
bclz_train_labels = bcolz.carray(rootdir= train_name+ '_labels.bclz', mode='r')

print(bclz_valid_data.shape, bclz_valid_labels.shape, type(bclz_valid_data))
print(bclz_test_data.shape, bclz_test_labels.shape, type(bclz_test_data)) 
print(bclz_train_data.shape, bclz_train_labels.shape, type(bclz_train_data)) 

(51000, 224, 224, 3) (51000, 3) <class 'bcolz.carray_ext.carray'>
(51000, 224, 224, 3) (51000, 3) <class 'bcolz.carray_ext.carray'>
(368586, 224, 224, 3) (368586, 3) <class 'bcolz.carray_ext.carray'>


In [5]:
# take percentage of data if required
bclz_valid_data3 = bclz_valid_data
bclz_test_data3 = bclz_test_data
bclz_train_data3 = bclz_train_data

bclz_valid_labels3 = bclz_valid_labels
bclz_test_labels3 = bclz_test_labels
bclz_train_labels3 = bclz_train_labels

valid_len = int(len(bclz_valid_data) * percent)
test_len = int(len(bclz_test_data) * percent)
train_len = int(len(bclz_train_data) * percent)
    
if percent < 1:
    bclz_valid_data3 = bclz_valid_data[:valid_len]
    bclz_test_data3 = bclz_test_data[:test_len]
    bclz_train_data3 = bclz_train_data[:train_len]

    bclz_valid_labels3 = bclz_valid_labels[:valid_len]
    bclz_test_labels3 = bclz_test_labels[:test_len]
    bclz_train_labels3 = bclz_train_labels[:train_len]
    
print(bclz_valid_data3.shape, bclz_valid_labels3.shape, type(bclz_valid_data3))
print(bclz_test_data3.shape, bclz_test_labels3.shape, type(bclz_test_data3))
print(bclz_train_data3.shape, bclz_train_labels3.shape, type(bclz_train_data3))

(255, 224, 224, 3) (255, 3) <class 'numpy.ndarray'>
(255, 224, 224, 3) (255, 3) <class 'numpy.ndarray'>
(1842, 224, 224, 3) (1842, 3) <class 'numpy.ndarray'>


In [12]:
valid_gen =bcolz_data_generator(bclz_valid_data3, bclz_valid_labels3, batch_size=batch_size, shuffle=True)
test_gen =bcolz_data_generator(bclz_test_data3, bclz_test_labels3, batch_size=batch_size, shuffle=True)
train_gen =bcolz_data_generator(bclz_train_data3, bclz_train_labels3, batch_size=batch_size, shuffle=True)

In [6]:
# experimental stuff
# Generate a model with all layers (with top)
vgg19 = VGG19(weights='imagenet', include_top=True)

x = Dropout(0.5,name="dropout1")(vgg19.layers[-3].output)
x = vgg19.layers[-2](x)
x = Dropout(0.5,name="dropout2")(x)
x = Dense(num_classes, activation='softmax', name='my_predictions')(x)

for layer in vgg19.layers:
    layer.trainable = False

my_model = Model(inputs=vgg19.input, outputs=x)

my_model.layers[-5].trainable = True
my_model.layers[-3].trainable = True
my_model.layers[-1].trainable = True

my_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [8]:
for layer in my_model.layers:
    if hasattr(layer, 'kernel_regularizer'):
        layer.kernel_regularizer= regularizers.l2(l2_weight_decay)

In [9]:
for layer in my_model.layers:
    if hasattr(layer, 'kernel_regularizer'):
        print(layer.name, layer.trainable,  layer.kernel_regularizer)
    else:
         print(layer.name, layer.trainable)

input_1 False
block1_conv1 False <keras.regularizers.L1L2 object at 0x7f03485fdac8>
block1_conv2 False <keras.regularizers.L1L2 object at 0x7f03485fde80>
block1_pool False
block2_conv1 False <keras.regularizers.L1L2 object at 0x7f03485fd860>
block2_conv2 False <keras.regularizers.L1L2 object at 0x7f03485fd898>
block2_pool False
block3_conv1 False <keras.regularizers.L1L2 object at 0x7f03485fd9e8>
block3_conv2 False <keras.regularizers.L1L2 object at 0x7f03485fdfd0>
block3_conv3 False <keras.regularizers.L1L2 object at 0x7f03485fd588>
block3_conv4 False <keras.regularizers.L1L2 object at 0x7f03485fd4e0>
block3_pool False
block4_conv1 False <keras.regularizers.L1L2 object at 0x7f03485fd9b0>
block4_conv2 False <keras.regularizers.L1L2 object at 0x7f03485fda20>
block4_conv3 False <keras.regularizers.L1L2 object at 0x7f03485fdd68>
block4_conv4 False <keras.regularizers.L1L2 object at 0x7f034860ce80>
block4_pool False
block5_conv1 False <keras.regularizers.L1L2 object at 0x7f034860c940>
bloc

In [12]:
for layer in my_model.layers:
    layer.trainable = True
    print(layer.name, layer.trainable)

input_1 True
block1_conv1 True
block1_conv2 True
block1_pool True
block2_conv1 True
block2_conv2 True
block2_pool True
block3_conv1 True
block3_conv2 True
block3_conv3 True
block3_conv4 True
block3_pool True
block4_conv1 True
block4_conv2 True
block4_conv3 True
block4_conv4 True
block4_pool True
block5_conv1 True
block5_conv2 True
block5_conv3 True
block5_conv4 True
block5_pool True
flatten True
fc1 True
dropout1 True
fc2 True
dropout2 True
my_predictions True


In [13]:
my_model.load_weights(model_path)
my_model.save_weights()

OSError: Unable to open file (Unable to open file: name = 'saved_models/test_all_layers_caffe_vgg19_weights.hdf5', errno = 2, error message = 'no such file or directory', flags = 0, o_flags = 0)

In [11]:
my_model.fit_generator(train_gen,
          steps_per_epoch= (1 + int(bclz_train_data3.shape[0] // batch_size)),
          epochs=epochs,
          validation_data=valid_gen,
          validation_steps= (1 + int(bclz_valid_data3.shape[0] // batch_size)),
          callbacks=[early_stopping, lrscheduler] )

NameError: name 'train_gen' is not defined

In [14]:
checkpointer = ModelCheckpoint(filepath=model_path, verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=2, verbose=1)
csv_logger = CSVLogger(loss_history_csv_name, append=True, separator=',')
lrscheduler = LearningRateScheduler(schedule=lr_schedule)

my_model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=lr, momentum=momentum),
              metrics=['accuracy'])
my_model.fit(bclz_train_data3, bclz_train_labels3,
          epochs=epochs,
          validation_data=(bclz_valid_data3, bclz_valid_labels3),
          callbacks=[early_stopping, lrscheduler])

Train on 1670 samples, validate on 208 samples
Epoch 1/15
1670/1670 [==============================] - 18s 11ms/step - loss: 1.0422 - acc: 0.7832 - val_loss: 0.6154 - val_acc: 0.8413
Epoch 2/15
1670/1670 [==============================] - 15s 9ms/step - loss: 0.2228 - acc: 0.9515 - val_loss: 0.4027 - val_acc: 0.9327
Epoch 3/15
1670/1670 [==============================] - 15s 9ms/step - loss: 0.0384 - acc: 0.9874 - val_loss: 0.5005 - val_acc: 0.9038
Epoch 4/15
1670/1670 [==============================] - 16s 9ms/step - loss: 0.0115 - acc: 0.9958 - val_loss: 0.4818 - val_acc: 0.9135
Epoch 00004: early stopping


In [ ]:
model.lo